In [1]:
# libraries needed 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import re
%matplotlib inline

# libraries we might not need
import csv

### Using the following code
to use the whole document you only need one file specified by filepath for the time being

In [2]:
%%time
# imports a random sample of size s from csv-file as a pandas dataframe
# pandas using python 3.X uses utf-8 encoding

# usage: specify file location, sample size and seed(used by random)
filepath = '../Data_git_ignore/FakeNewsCorpus_250_clean.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 250                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

# init dataframe with specified values
df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

# visual output
#print(df.shape, '<- size of dataframe \n')
df.head()

Wall time: 20 ms


,Unnamed: 0.1,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,content_tokens,type_id
168,168,26536,alternet.org,political,https://www.alternet.org/activism/if-we-dont-a...,if we dont act now fascism will be on our door...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"If We Don't Act Now, Fascism Will Be on Our Do...",NaN,NaN,NaN,Timothy Snyder warns: History gives us a bunch...,NaN,NaN,[],6
37,37,6045,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-conservative/...,greg hunter big banks in big trouble syrianort...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Greg Hunter: Big Banks in Big Trouble, Syria/N...",John Rolls,NaN,NaN,NaN,NaN,NaN,[],3
212,212,34827,beforeitsnews.com,fake,http://beforeitsnews.com/survival/2017/11/woma...,woman who flipped off trumps motorcade was fir...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Woman Who Flipped Off Trump’s Motorcade Was Fi...,NaN,NaN,NaN,NaN,NaN,NaN,[],3
143,143,23066,beforeitsnews.com,fake,http://beforeitsnews.com/spirit/2015/06/mary-m...,mary magdalene healing shame around addictions...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,MARY MAGDALENE: Healing Shame Around Addiction...,Mercedes Kirkel,NaN,NaN,NaN,NaN,NaN,[],3
177,177,27309,beforeitsnews.com,fake,http://beforeitsnews.com/healthcare/2014/09/ca...,cancer causes\r \r of readers think this stor...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cancer causes,Caroline'S Breast Cancer Blog,NaN,NaN,NaN,NaN,NaN,[],3


In [3]:
df['id'] = pd.to_numeric(df['id'], errors = 'coerce', downcast = 'integer')
df.drop_duplicates(subset = 'id', inplace = True)

In [4]:
df = df.dropna(subset=['id']).set_index('id')
df.shape

(250, 17)

### data-tables: [name]-uniq / relational-tables: [name]_in
creating csv-files for database

In [5]:
# specify where to save all csv-files
path = '../Data_git_ignore/clean_csv/'

# create temporary dataframe and use article id as index 
out_df = pd.DataFrame({'id':df.index})
out_df.set_index('id', inplace=True)

In [6]:
def generateUniqe (category):
    type_array = df[category].unique() # get array of unique types
    type_df = pd.DataFrame({'id': np.arange(type_array.size), 'name':type_array})

    # write file and free memory
    type_df.to_csv(path + category + '_clean.csv', index=False, header=True)
    #del type_array
    #del type_df # tmp delete later

    # create dict with type_name as key - [swap type with type_id]
    type_name_as_key_df = type_df.set_index('name')
    type_dict = type_name_as_key_df['id'].to_dict()

    # replace type with tag id and create new column
    type_id = np.array([type_dict[key] for key in df[category].to_numpy()])
    #df['type_id'] =type_id
    
    return type_id

def generateUniqeArray (category):
    # creates list of list but formaly it is a pd.series of lists
    tags_series_of_lists = df[category].dropna().str.split(', ') # -> ', ' not ','

    if not category in out_df: ### tmp need another method ###
        out_df.insert(0,column = category, value = tags_series_of_lists)

    # flattern tags_series_of_lists to a set(ie. unique values only)
    tags_list = list(set([item for sublist in tags_series_of_lists for item in sublist]))

    # create dataframe
    tags_df = pd.DataFrame({'id': np.arange(len(tags_list)), 'name':tags_list})

    # write file and free memory
    tags_df.to_csv(path + category + '_clean.csv', index=False, header=True)
    del tags_series_of_lists
    del tags_list
    
    # generate the relation
    # get all pairs of article_id and tags in a article (for all articles)
    articles_id_tags_name_pairs_df = out_df[category].dropna().explode().drop_duplicates(keep = 'first')

    # split tags_name and articles_id
    articles_id_array = articles_id_tags_name_pairs_df.index.to_numpy()
    tags_name_array = articles_id_tags_name_pairs_df.to_numpy()

    # create dict with tag_name as key - [swap tags with tags_id]
    tags_name_as_key_df = tags_df.set_index('name')
    tags_dict = tags_name_as_key_df['id'].to_dict()

    # replace tags with tag id
    tags_id = np.array([tags_dict[key] for key in tags_name_array])

    # create dataframe
    tags_in_df = pd.DataFrame(data=articles_id_array, index=tags_id, columns=['article_id'])
    tags_in_df.index.name='tags_id'

    # write file and free memory
    tags_in_df.to_csv(path + category + '_in.csv', index=True, header=True)

In [7]:
### type_clean - data-table ###
df["type"] = generateUniqe("type")

In [8]:
### tags_clean & tags_in - data-table ###
generateUniqeArray("tags")

In [9]:
### authors_clean & authors_in - data-table ###
generateUniqeArray("authors")

In [10]:
### domain_clean - data-table ###
df["domain"] = generateUniqe("domain")

In [11]:
### meta_keywords_clean & meta_keywords_in - data-table ###
generateUniqeArray("meta_keywords")

In [12]:
### article clean ###

df[['domain', 'type', "url", "content", "title", "meta_description", "scraped_at",  "updated_at", "inserted_at"]].to_csv(path + 'article_clean.csv', header=True)

In [13]:
### Setting up wikinews

In [14]:
# imports a random sample of size s from csv-file as a pandas dataframe
# pandas using python 3.X uses utf-8 encoding

# usage: specify file location, sample size and seed(used by random)
filepath = '../Data_sample/wiki_news_raw_backup.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 1000                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

# init dataframe with specified values
df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

# visual output
#print(df.shape, '<- size of dataframe \n')
df.head()

,categories,content,publish_date,scraped_at,source_urls,sources_domain_urls,title
article_url,,,,,,,
https://en.wikinews.org/wiki/Namibia_votes_in_presidential_election,"November 28, 2009,Namibia,Published,Elections,...",\r\n Voters are heading to the polls today in ...,"November 28, 2009",2020-04-23,http://www1.voanews.com/english/news/africa/27...,"/wiki/VOA_News,/wiki/Associated_Press",Namibia votes in presidential election
https://en.wikinews.org/wiki/Scientology_defector_arrested_after_attempting_to_leave_organization,"October 9, 2010,Religion,Scientology,Published...",\r\n Recent reports have revealed that a defec...,"October 9, 2010",2020-04-22,http://newsfeed.time.com/2010/10/09/scientolog...,"/wiki/TIME,https://en.wikipedia.org/wiki/LA_We...",Scientology defector arrested after attempting...
https://en.wikinews.org/wiki/Nobel_laureate_Doris_Lessing:_9/11_%27wasn%27t_that_terrible%27_compared_to_IRA_attacks,"October 22, 2007,Published,Archived,United Kin...","\r\n British writer Doris Lessing , recipien...","October 22, 2007",2020-04-22,http://www.ctv.ca/servlet/ArticleNews/story/CT...,"https://en.wikipedia.org/wiki/CTV.ca,https://e...",Nobel laureate Doris Lessing: 9/11 'wasn't tha...
https://en.wikinews.org/wiki/No-fly_zone_demanded_by_Syrian_protesters,"October 29, 2011,Published,Archived,Syria,Midd...",\r\n Friday saw protests across Syria callin...,"October 29, 2011",2020-04-22,http://www.bbc.co.uk/news/world-middle-east-15...,"/wiki/BBC_News_Online,/wiki/Agence_France-Pres...",No-fly zone demanded by Syrian protesters
https://en.wikinews.org/wiki/Romanian_oil_company_claims_Iran_has_seized_one_of_its_rigs,"August 22, 2006,Published,AutoArchived,World,P...",\r\n An Orizont oil platform in the Persian G...,"August 22, 2006",2020-04-22,http://www.bloomberg.com/apps/news?pid=2060108...,"https://en.wikipedia.org/wiki/Bloomberg,/wiki/...",Romanian oil company claims Iran has seized on...


In [24]:
df['id'] = pd.to_numeric(range(10000000, 10000000+df.shape[0]), errors = 'coerce', downcast = 'integer')
df.drop_duplicates(subset = 'id', inplace = True)

df = df.dropna(subset=['id']).set_index('id')
df.shape

(1000, 7)

In [18]:
# create temporary dataframe and use article id as index 
out_df = pd.DataFrame({'id':df.index})
out_df.set_index('id', inplace=True)

In [19]:
### categoires_clean & categories_in ###
generateUniqeArray("categories")

In [20]:
### sources_domain_urls_clean & sources_domain_urls_in
generateUniqeArray("sources_domain_urls")

In [26]:
### article clean ###
df[['content', "title", "scraped_at", "publish_date"]].to_csv(path + 'wiki_article_clean.csv', header=True)

In [22]:
df.head()

,categories,content,publish_date,scraped_at,source_urls,sources_domain_urls,title
id,,,,,,,
10000000,"November 28, 2009,Namibia,Published,Elections,...",\r\n Voters are heading to the polls today in ...,"November 28, 2009",2020-04-23,http://www1.voanews.com/english/news/africa/27...,"/wiki/VOA_News,/wiki/Associated_Press",Namibia votes in presidential election
10000001,"October 9, 2010,Religion,Scientology,Published...",\r\n Recent reports have revealed that a defec...,"October 9, 2010",2020-04-22,http://newsfeed.time.com/2010/10/09/scientolog...,"/wiki/TIME,https://en.wikipedia.org/wiki/LA_We...",Scientology defector arrested after attempting...
10000002,"October 22, 2007,Published,Archived,United Kin...","\r\n British writer Doris Lessing , recipien...","October 22, 2007",2020-04-22,http://www.ctv.ca/servlet/ArticleNews/story/CT...,"https://en.wikipedia.org/wiki/CTV.ca,https://e...",Nobel laureate Doris Lessing: 9/11 'wasn't tha...
10000003,"October 29, 2011,Published,Archived,Syria,Midd...",\r\n Friday saw protests across Syria callin...,"October 29, 2011",2020-04-22,http://www.bbc.co.uk/news/world-middle-east-15...,"/wiki/BBC_News_Online,/wiki/Agence_France-Pres...",No-fly zone demanded by Syrian protesters
10000004,"August 22, 2006,Published,AutoArchived,World,P...",\r\n An Orizont oil platform in the Persian G...,"August 22, 2006",2020-04-22,http://www.bloomberg.com/apps/news?pid=2060108...,"https://en.wikipedia.org/wiki/Bloomberg,/wiki/...",Romanian oil company claims Iran has seized on...
